In [ ]:
from fastai.tabular.all import *

In [ ]:
input_path = "../input/tabular-playground-series-jun-2021/"
train = pd.read_csv(input_path + "train.csv")

In [ ]:
train

In [ ]:
train.nunique().sort_values(ascending=False)[:20]

In [ ]:
train.info()

In [ ]:
train = df_shrink(train)

In [ ]:
train.drop("id", axis=1, inplace=True)

# Preprocessing

split the data set in a stratified manner

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
splits = next(stratified_split.split(train.iloc[:,1:], train["target"]))
splits_list = [array.tolist() for array in splits]

preprocess our data using fastai's `TabularPandas`

In [ ]:
cont,cat = cont_cat_split(train, 105, dep_var="target")
procs = [Categorify]

to = TabularPandas(train, procs, cat, cont, y_names="target", splits=splits_list)

form dataloaders for training and validation set

In [ ]:
dls = to.dataloaders(1024)

## Neural Net

In [ ]:
learn = tabular_learner(dls, layers=[200, 100])
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3, 1e-4, wd=0.8)

# Predicting

In [ ]:
test = pd.read_csv(input_path + "test.csv")
to_tst = to.new(test)
to_tst.process()

prediction of neural net

In [ ]:
tst_dl = dls.valid.new(to_tst)
preds, _ = learn.get_preds(dl=tst_dl)

In [ ]:
sub = pd.read_csv(input_path + "sample_submission.csv")
sub.iloc[:, 1:] = preds
sub.to_csv("submission.csv", index=False)